<a href="https://colab.research.google.com/github/rajchoksi1997/CMPE257-Project/blob/main/Main_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>